In [ ]:
#version 1.0.1
!pip install -q gradio
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
def lnreg(x,y,xcol,ycol,f2):
  reg = linear_model.LinearRegression()
  x_train=x.reshape(-1,1)
  y_train=y.reshape(-1,1)
  fitting=reg.fit(x_train,y_train)
  df1=pd.read_csv(f2)
  x_test=np.array(df1[xcol])
  y_test=np.array(df1[ycol])
  x_test=x_test.reshape(-1,1)
  y_test=y_test.reshape(-1,1)
  predict=reg.predict(x_test)
  rmse = np.sqrt(mean_squared_error(y_test,predict))
  r_squared = r2_score(y_test,predict)
  return str(rmse),str(r_squared)

def linregtest(f1):
  def updownsplitTrend(xsplit,ysplit,x,y):
      l=xsplit
      xmid=int(xsplit/2)
      ymid=int(ysplit/2)
      i=0
      inc=0
      dec=0
      while (xsplit>-1 and ysplit>-1) and i<l-1:
          if (x[i]<x[xsplit-1] and y[i]<y[ysplit-1]) or (x[i+1]<x[xsplit-2] and y[i+1]<y[ysplit-2]) or (x[i+2]<x[xsplit-3] and y[i+2]<y[ysplit-3]):
              if (x[xmid-4]<x[xmid+3] and y[ymid-4]<y[ymid+3]) or (x[xmid-3]<x[xmid+2] and y[ymid-3]<y[ymid+2]) or (x[xmid-2]<x[xmid+1] and y[ymid-2]<y[ymid+1]) or (x[xmid-1]<x[xmid] and y[ymid-1]<y[ymid]):
                  inc+=1
          elif (x[i]>x[xsplit-1] and y[i]>y[ysplit-1]) or (x[i+1]>x[xsplit-2] and y[i+1]>y[ysplit-2]) or (x[i+2]>x[xsplit-3] and y[i+2]>y[ysplit-3]):
              if (x[xmid-4]>x[xmid+3] and y[ymid-4]>y[ymid+3]) or (x[xmid-3]>x[xmid+2] and y[ymid-3]>y[ymid+2]) or (x[xmid-2]>x[xmid+1] and y[ymid-2]>y[ymid+1]) or (x[xmid-1]>x[xmid] and y[ymid-1]>y[ymid]):
                  dec+=1
          xsplit-=1
          ysplit-=1
      if inc>0:
          print("Increasing Trend")
          t=True
      elif dec>0:
          print("Decreasing Trend")
          t=True
      else:
          print("Inconsistent updown trend")
          t=False
      return t
          
  def splitTrend(x,y,xsplit,ysplit):
      up=0
      down=0
      updown=0
      negativecorrelation=0
      upstreak=0
      downstreak=0
      ncstreak=0
      l1=len(x)
      l2=len(y)
      l=l1
      i=0
      while l>0 and i<(l1-1):
      #check dtype of x and y when returning from df then get into streaks
          if (x[i+1]>x[i] and y[i+1]>y[i]):
              if i>=1:
                  upstreak+=1
                  up+=1
                  downstreak=0
                  ncstreak=0
          elif (x[i+1]<x[i] and y[i+1]<y[i]) :
              negativecorrelation+=1
              ncstreak+=1
              up+=1
              upstreak=0
              downstreak=0
          elif (x[i+1]>x[i] and y[i+1]<y[i]) :
              negativecorrelation+=1
              ncstreak+=1
              down+=1
              upstreak=0
              downstreak=0
          elif  (x[i+1]<x[i] and y[i+1]>y[i]):
              if i>=1:
                  downstreak+=1
                  down+=1
                  upstreak=0
          if i==(l1-2):
              t=trend(up,down,updown,negativecorrelation,l1,l2,x,y,xsplit,ysplit)
          l=l-1
          i=i+1
      print(up)
      print(down)
      print(negativecorrelation)
      return t

  def Range(n,p,k):
      count=0
      if n%2==0:
          if n<10000:
              k=n/10
              p=k/10
          elif n>10000:
              count+=1
              n=n/2
              Range(n,0,0)
      else:
          count-=1
          n=n-1
          Range(n,0,0)
      return int(n),int(p),int(k),int(count)

  def updowntrend(xlen,ylen,xsplit,ysplit):
      while (xlen==ylen and xlen>0 and ylen>0):
          if xlen%2==0:
              if xlen>500:
                  xlen=xlen/2
                  ylen=ylen/2
                  xsplit=xlen/10
                  ysplit=ylen/10
              else:
                  xsplit=xlen/10
                  ysplit=ylen/10
          else:
              xlen-=1
              ylen-=1
              updowntrend(xlen,ylen,xsplit,ysplit)
      return xsplit,ysplit
      

  def trend(up,down,updown,negativecorrelation,l1,l2,x,y,xsplit,ysplit):
      if (up-down)in [-1,0,1]:
          updown+=1
      elif (up-negativecorrelation) in [-1,0,1]:
          updown+=1
      if updown>0:
          # print("updown trend exists")
          if l1<100:
              updownsplitTrend(l1,l2)
          else:
              xsplit,ysplit=updowntrend(l1,l2,xsplit,ysplit)
              t=updownsplitTrend(xsplit,ysplit,x,y)
      elif up>(l1/2):
              print("Plain Increasing trend")
              t=True
      elif down>(l1):
              print("Plain Decreasing trend")
              t=True
      else:
          print("No co-realtion")
          t=False
      print(updown)
      return t

  df=pd.read_csv(f1)
  n=df.shape[0]
  k=0
  p=0
  if n>10:
      n1,p,k,c=Range(n,p,k)
  print(n1,p,k,c)

  col=list(df.columns)
  rcol=[]
  for a in col:
      if df[a].dtype==int or df[a].dtype==float or df[a].dtype==np.int64 or df[a].dtype==np.float64 or df[a].dtype==np.int32 or df[a].dtype==np.float32:
          rcol.append(a)
  def call(rcol,k,p,df,xsplit,ysplit):
      i=0
      p1=0
      tval=0 
      ln=len(rcol)
      # print(ln)
      while i<(ln-1):
          colx=rcol[i]
          coly=rcol[i+1]
          for j in range(k):
              if j==0:
                  x=df[colx[:p-1]]
                  y=df[coly[:p-1]]
                  p1=p+p
              else:
                  x=df[colx[p:p1-1]]
                  y=df[coly[p:p1-1]]
                  p=p1
                  p1=p+p    
              t=splitTrend(x,y,xsplit,ysplit)
              if t==True:
                  tval+=1
              if tval>=1:
                  break
              else:
                return '',''
          if tval>=1:
              break
          i+=1       
      return colx,coly

  xsplit=0
  ysplit=0
  xcol,ycol=call(rcol,k,p,df,xsplit,ysplit)
  if xcol=='' and ycol=='':
    return False,xcol,ycol,[],[]
  else:
    x=df.loc[:,xcol]
    y=df.loc[:,ycol]
    return True,xcol,ycol,np.array(x),np.array(y)

import gradio as gr
def linreg(file1,file2):
  v,xcol,ycol,x,y=linregtest(file1.name)
  if v==True:
    rmse,rsq=lnreg(x,y,xcol,ycol,file2.name)
    return "RMSE:"+rmse+"\nR-SQUARED:"+rsq
  else:
    return "0"

inf=gr.Interface(linreg,inputs=["file","file"],outputs="text",capture_session=True)
inf.launch(share=True)
#tester code
# def linreg(file1,file2):
#   v,xcol,ycol,x,y=linregtest(file1)
#   if v==True:
#     rmse,rsq=lnreg(x,y,xcol,ycol,file2)
#     return "RMSE:"+rmse+"R-SQUARED:"+rsq
#   else:
#     return "0"
# linreg("train.csv","test.csv")

/usr/local/lib/python3.7/dist-packages/gradio/interface.py:236: UserWarning: The `capture_session` parameter in the `Interface` is deprecated and may be removed in the future.
  "The `capture_session` parameter in the `Interface`"


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://46328.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7fd64f6a0ad0>,
 'http://127.0.0.1:7862/',
 'https://46328.gradio.app')

In [ ]:
#version 1.0.2
!pip install -q gradio
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
def lnreg(x,y,xcol,ycol,f2):
  reg = linear_model.LinearRegression()
  x_train=x.reshape(-1,1)
  y_train=y.reshape(-1,1)
  fitting=reg.fit(x_train,y_train)
  df1=pd.read_csv(f2)
  x_test=np.array(df1[xcol])
  y_test=np.array(df1[ycol])
  x_test=x_test.reshape(-1,1)
  y_test=y_test.reshape(-1,1)
  predict=reg.predict(x_test)
  rmse = np.sqrt(mean_squared_error(y_test,predict))
  r_squared = r2_score(y_test,predict)
  return rmse,r_squared
def linregtest(f1,f2):
  def updownsplitTrend(xsplit,ysplit,x,y):
      l=xsplit
      xmid=int(xsplit/2)
      ymid=int(ysplit/2)
      i=0
      inc=0
      dec=0
      while (xsplit>3 and ysplit>3) and i<l-1:
          if (x[i]<x[xsplit-1] and y[i]<y[ysplit-1]) or (x[i+1]<x[xsplit-2] and y[i+1]<y[ysplit-2]) or (x[i+2]<x[xsplit-3] and y[i+2]<y[ysplit-3]):
              if (x[xmid-4]<x[xmid+3] and y[ymid-4]<y[ymid+3]) or (x[xmid-3]<x[xmid+2] and y[ymid-3]<y[ymid+2]) or (x[xmid-2]<x[xmid+1] and y[ymid-2]<y[ymid+1]) or (x[xmid-1]<x[xmid] and y[ymid-1]<y[ymid]):
                  inc+=1
          elif (x[i]>x[xsplit-1] and y[i]>y[ysplit-1]) or (x[i+1]>x[xsplit-2] and y[i+1]>y[ysplit-2]) or (x[i+2]>x[xsplit-3] and y[i+2]>y[ysplit-3]):
              if (x[xmid-4]>x[xmid+3] and y[ymid-4]>y[ymid+3]) or (x[xmid-3]>x[xmid+2] and y[ymid-3]>y[ymid+2]) or (x[xmid-2]>x[xmid+1] and y[ymid-2]>y[ymid+1]) or (x[xmid-1]>x[xmid] and y[ymid-1]>y[ymid]):
                  dec+=1
          xsplit-=1
          ysplit-=1
      if inc>0:
          # "Increasing Trend"
          t=True
      elif dec>0:
          # "Decreasing Trend"
          t=True
      else:
          # "Inconsistent updown trend"
          t=False
      return t
          
  def splitTrend(x,y,xsplit,ysplit):
      up=0
      down=0
      updown=0
      negativecorrelation=0
      upstreak=0
      downstreak=0
      ncstreak=0
      l1=len(x)
      l2=len(y)
      l=l1
      i=0
      while l>0 and i<(l1-1):
      #check dtype of x and y when returning from df then get into streaks
          if (x[i+1]>x[i] and y[i+1]>y[i]):
              if i>=1:
                  upstreak+=1
                  up+=1
                  downstreak=0
                  ncstreak=0
          elif (x[i+1]<x[i] and y[i+1]<y[i]) :
              negativecorrelation+=1
              ncstreak+=1
              up+=1
              upstreak=0
              downstreak=0
          elif (x[i+1]>x[i] and y[i+1]<y[i]) :
              negativecorrelation+=1
              ncstreak+=1
              down+=1
              upstreak=0
              downstreak=0
          elif  (x[i+1]<x[i] and y[i+1]>y[i]):
              if i>=1:
                  downstreak+=1
                  down+=1
                  upstreak=0
          
          if i==(l1-2):
              t=trend(up,down,updown,negativecorrelation,l1,l2,x,y,xsplit,ysplit)
          l=l-1
          i=i+1
      return t

  def Range(n,p,k):
      count=0
      if n%2==0:
          if n<10000:
              k=n/10
              p=k/10
          elif n>10000:
              count+=1
              n=n/2
              Range(n,0,0)
      else:
          count-=1
          n=n-1
          Range(n,0,0)
      return int(n),int(p),int(k),int(count)

  def updowntrend(xlen,ylen,xsplit,ysplit):
      while (xlen==ylen and xlen>0 and ylen>0):
          if xlen%2==0:
              if xlen>500:
                  xlen=xlen/2
                  ylen=ylen/2
                  xsplit=xlen/10
                  ysplit=ylen/10
                  break
              else:
                  xsplit=xlen/10
                  ysplit=ylen/10
                  break
          else:
              xlen-=1
              ylen-=1
              updowntrend(xlen,ylen,xsplit,ysplit)
      return xsplit,ysplit
      

  def trend(up,down,updown,negativecorrelation,l1,l2,x,y,xsplit,ysplit):
      if (up-down)in [-1,0,1]:
          updown+=1
      elif (up-negativecorrelation) in [-1,0,1]:
          updown+=1
      if updown>0:
          # "updown trend exists"
          if l1<100:
              t=updownsplitTrend(l1,l2,x,y)
          else:
              xsplit,ysplit=updowntrend(l1,l2,xsplit,ysplit)
              t=updownsplitTrend(xsplit,ysplit,x,y)
      elif up>(l1/2):
              # "Plain Increasing trend"
              t=True
      elif down>(l1):
              # "Plain Decreasing trend"
              t=True
      else:
          # "No co-realtion"
          t=False
      return t

  df=pd.read_csv(f1)
  n=df.shape[0]
  k=0
  p=0
  if n>10:
      n1,p,k,c=Range(n,p,k)

  col=list(df.columns)
  rcol=[]
  for a in col:
      if df[a].dtype==int or df[a].dtype==float or df[a].dtype==np.int64 or df[a].dtype==np.float64 or df[a].dtype==np.int32 or df[a].dtype==np.float32:
          rcol.append(a)
  def call(rcol,df,xsplit,ysplit,f2):
      i=0
      tval=0 
      ln=len(rcol)
      rmsescore=[]
      rsqscore=[]
      colpair=[]
      rmseind=0
      rmse=0
      rsq=0
      while i<(ln-1):
          colx=rcol[i]
          coly=rcol[i+1]
          x=df[colx]
          y=df[coly]
          t=splitTrend(x,y,xsplit,ysplit)
          if t==True:
            tval+=1
          if tval>=1:
            rmse,rsq=lnreg(np.array(x),np.array(y),colx,coly,f2)
            rmsescore.append(rmse)
            rsqscore.append(rsq)
            colpair.append([colx,coly])
          else:
            return '',''
          i+=1
          
      for a in rmsescore:
          if min(rmsescore)==a:
              cols=colpair[rmseind]
              colx,coly=cols[0],cols[1]
          rmseind+=1 
      return colx,coly

  xsplit=0
  ysplit=0
  xcol,ycol=call(rcol,df,xsplit,ysplit,f2)
  if xcol=='' and ycol=='':
    return False,xcol,ycol,[],[]
  else:
    x=df.loc[:,xcol]
    y=df.loc[:,ycol]
    return True,xcol,ycol,np.array(x),np.array(y)

import gradio as gr
def linreg(file1,file2):
  v,xcol,ycol,x,y=linregtest(file1.name,file2.name)
  if v==True:
    rmse,rsq=lnreg(x,y,xcol,ycol,file2.name)
    return "RMSE:"+str(rmse)+"\nR-SQUARED:"+str(rsq)
  else:
    return "0"

inf=gr.Interface(linreg,inputs=["file","file"],outputs="text",capture_session=True)
inf.launch(share=True)

#tester code
# def linreg(file1,file2):
#   v,xcol,ycol,x,y=linregtest(file1,file2)
#   if v==True:
#     rmse,rsq=lnreg(x,y,xcol,ycol,file2)
#     return "RMSE:"+str(rmse)+"R-SQUARED:"+str(rsq)
#   else:
#     return "0"
# linreg("train.csv","test.csv")

     |████████████████████████████████| 2.9 MB 5.8 MB/s 
     |████████████████████████████████| 84 kB 2.6 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 1.1 MB 52.2 MB/s 
     |████████████████████████████████| 211 kB 63.4 MB/s 
     |████████████████████████████████| 253 kB 42.8 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 2.0 MB 26.2 MB/s 
     |████████████████████████████████| 271 kB 49.1 MB/s 
     |████████████████████████████████| 144 kB 42.4 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 
     |████████████████████████████████| 10.9 MB 39.5 MB/s 
     |████████████████████████████████| 58 kB 4.9 MB/s 
     |████████████████████████████████| 79 kB 5.8 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 3.6 MB 45.8 MB/s 
     |████████████████████████████████| 61 kB 198 kB/s 
     |████████████████████████

/usr/local/lib/python3.7/dist-packages/gradio/interface.py:248: UserWarning: The `capture_session` parameter in the `Interface` is deprecated and may be removed in the future.
  "The `capture_session` parameter in the `Interface`"


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://51525.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f00b49657d0>,
 'http://127.0.0.1:7860/',
 'https://51525.gradio.app')